In [81]:
import numpy as np
import torch

In [82]:
# set logger and enforce reproducibility
from GPErks.log.logger import get_logger
from GPErks.utils.random import set_seed
log = get_logger()
seed = 8
set_seed(seed)  # reproducible sampling

**2D function example**: Currin et al. (1988)

$f(x) = \left[1 - \exp{\left(1 - \dfrac{1}{2 x_2}\right)}\right]\,\left(\dfrac{2300 x_{1}^3 + 1900 x_{1}^2 + 2092 x_{1} + 60}{100 x_{1}^3 + 500 x_{1}^2 + 4 x_{1} + 20}\right)$

In [83]:
# function to learn (normally a high-dimensional, expensive deterministic model)
from GPErks.utils.test_functions import currin_exp
f = lambda X: np.array([currin_exp(x) for x in X])
D = 2

In [84]:
# build dataset
from GPErks.gp.data.dataset import Dataset
dataset = Dataset.build_from_function(
    f,
    D,
    n_train_samples=20,
    n_test_samples=25,
    design="lhs",
    seed=seed,
)

In [85]:
# choose likelihood
from gpytorch.likelihoods import GaussianLikelihood
likelihood = GaussianLikelihood()

In [86]:
# choose mean function
from gpytorch.means import LinearMean
mean_function = LinearMean(input_size=dataset.input_size)

In [87]:
# choose kernel
from gpytorch.kernels import RBFKernel, ScaleKernel
kernel = ScaleKernel(RBFKernel(ard_num_dims=dataset.input_size))

In [88]:
# choose metrics
from torchmetrics import MeanSquaredError, R2Score
metrics = [MeanSquaredError(), R2Score()]

In [89]:
# define experiment
from GPErks.gp.experiment import GPExperiment
experiment = GPExperiment(
    dataset,
    likelihood,
    mean_function,
    kernel,
    n_restarts=3,
    metrics=metrics,
    seed=seed  # reproducible training
)

In [90]:
# dump experiment in config file
config_file = "example_3.ini"
experiment.save_to_config_file(config_file)

In [91]:
# load experiment from config file
from GPErks.gp.experiment import load_experiment_from_config_file
experiment = load_experiment_from_config_file(
    config_file,
    dataset  # notice that we still need to provide the dataset used!
)

In [92]:
# choose training options: device + optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"
optimizer = torch.optim.Adam(experiment.model.parameters(), lr=0.1)

In [93]:
# train model
from GPErks.train.emulator import GPEmulator
emulator = GPEmulator(experiment, device)

In [94]:
# snapshotting
from GPErks.serialization.path import posix_path
from GPErks.train.snapshot import (
    EveryEpochSnapshottingCriterion,
    EveryNEpochsSnapshottingCriterion,
    NeverSaveSnapshottingCriterion
)
import os

snapshot_dir = posix_path(os.getcwd(), "snapshot", "example_3")
train_restart_template = "restart_{restart}"
train_epoch_template = "epoch_{epoch}.pth"

snapshot_file = train_epoch_template
snpc = EveryEpochSnapshottingCriterion(
    posix_path(snapshot_dir, train_restart_template),
    snapshot_file
)

# training
emulator.train(optimizer, snapshotting_criterion=snpc)

INFO:2021-08-04 17:11:24,736:emulator:train:L83: Training emulator...
INFO:2021-08-04 17:11:24,739:emulator:train:L99: Running restart 1...
/home/gianvito/personal/GPErks/venv/lib/python3.7/site-packages/gpytorch/models/exact_gp.py:284: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  GPInputWarning,
INFO:2021-08-04 17:11:24,764:emulator:_train_once:L308: [  1/100] Training Loss: 0.8316 - MeanSquaredError: 0.0849 - R2Score: 0.9151
INFO:2021-08-04 17:11:24,775:emulator:_train_once:L308: [  2/100] Training Loss: 0.7894 - MeanSquaredError: 0.0816 - R2Score: 0.9184
INFO:2021-08-04 17:11:24,787:emulator:_train_once:L308: [  3/100] Training Loss: 0.7519 - MeanSquaredError: 0.0795 - R2Score: 0.9205
INFO:2021-08-04 17:11:24,798:emulator:_train_once:L308: [  4/100] Training Loss: 0.7177 - MeanSquaredError: 0.0780 - R2Score: 0.9220
INFO:2021-08-04 17:11:24,810:emulator:_train_once:L308: [  5/100] Training Loss: 0.6858 - MeanSquaredError: 0.0767 


Bias: 1.2667
Weights: tensor([-0.0557, -2.5851], device='cuda:0')
Outputscale: 0.1248
Lengthscales: tensor([0.1160, 2.2744], device='cuda:0')
Likelihood noise: 0.0262


(OrderedDict([('likelihood.noise_covar.raw_noise', tensor([-3.6342])),
              ('mean_module.weights',
               tensor([[-0.0557],
                       [-2.5851]])),
              ('mean_module.bias', tensor([1.2667])),
              ('covar_module.raw_outputscale', tensor(-2.0184)),
              ('covar_module.base_kernel.raw_lengthscale',
               tensor([[-2.0959,  2.1658]]))]),
 <GPErks.train.train_stats.TrainStats at 0x7f8d672fc9d0>)

In [95]:
# inference on stored test set
from GPErks.perks.inference import Inference
inference = Inference(emulator)
inference.summary()

                   Score
MeanSquaredError  0.7254
R2Score           0.9013


In [96]:
# loading emulator
best_model_file = posix_path(
    snapshot_dir,
    "best_model.pth"
)
best_model_state = torch.load(best_model_file, map_location=torch.device(device))

emulator1 = GPEmulator(experiment, device)
emulator1.model.load_state_dict(best_model_state)

<All keys matched successfully>

In [97]:
inference = Inference(emulator1)
inference.summary()

                   Score
MeanSquaredError  0.7254
R2Score           0.9013
